In [1]:
from pyspark.sql import SparkSession
import numpy as np

In [2]:
spark = SparkSession.builder.appName("frames").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/23 22:45:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
events = spark.read.csv('Data/events.csv', header=True, inferSchema=True,sep=';')

In [4]:
import pp_events as pp

In [5]:
events_shot = pp.preprocessing(events)

In [6]:
events_shot.columns

['id',
 'player_id',
 'shot_location_x',
 'shot_location_y',
 'distance_to_goal',
 'shot_angle',
 'preferred_foot_shot',
 'under_pressure',
 'shot_aerial_won',
 'shot_first_time',
 'shot_one_on_one',
 'shot_open_goal',
 'shot_follows_dribble',
 'shot_statsbomb_xg',
 'shot_outcome',
 'goal']

In [7]:
df = pp.shot_frame_to_df(events,spark)

In [8]:
df.columns

Index(['Shot_id', 'x', 'y', 'position', 'teammate'], dtype='object')

In [9]:
import pandas as pd

# Assuming your DataFrame is called `df`
# Group by `Shot_id` and aggregate the x and y values as pairs
result = (
    df.groupby('Shot_id')
    .apply(lambda group: group[['x', 'y']].values.tolist())
    .reset_index(name='coordinates'))

result

/tmp/ipykernel_22123/3126286815.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: group[['x', 'y']].values.tolist())


,Shot_id,coordinates
0,00018844-2ac3-441c-b751-8ed47ad26daf,"[[107.4, 43.0], [101.2, 44.9], [103.6, 39.5], ..."
1,00019fe2-3293-4936-9af6-ee6809f64abf,"[[94.9, 53.1], [117.6, 39.7], [95.0, 48.3], [9..."
2,0002e561-40cf-401a-a298-61f1cce6a57c,"[[103.8, 36.3], [110.1, 33.7], [103.3, 38.2], ..."
3,0003898b-7b6e-4165-ad8a-d9f64e6c8213,"[[118.5, 25.4], [120.0, 23.3], [109.7, 32.5], ..."
4,0003d15d-08d9-4b69-87c7-bd835c75f9c0,"[[106.4, 31.2], [107.2, 41.1], [116.3, 37.8], ..."
...,...,...
60665,fffbcf63-e773-4f52-947c-c90b7db5d4df,"[[101.7, 58.5], [80.8, 16.7], [100.1, 7.9], [1..."
60666,fffbda66-f7f4-4552-86be-d02dbb05c8d4,"[[93.2, 56.0], [118.3, 44.2], [113.3, 48.8], [..."
60667,fffc3716-8b14-4231-8d05-71f902d57dc4,"[[98.2, 56.7], [99.5, 39.7], [86.0, 19.8], [93..."
60668,fffd40df-4fd8-4f18-a97b-98db237d949f,"[[98.1, 59.6], [105.9, 41.9], [104.2, 35.4], [..."


In [10]:
frames = spark.createDataFrame(result)

In [11]:
# join events_shot with frames
frames = frames.join(events_shot.select('id','shot_location_x','shot_location_y'),frames.Shot_id == events_shot.id).drop('id')

In [12]:
frames.show()

24/12/23 22:46:58 WARN TaskSetManager: Stage 3 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+---------------+---------------+
|             Shot_id|         coordinates|shot_location_x|shot_location_y|
+--------------------+--------------------+---------------+---------------+
|00d83f31-e271-4a1...|[[96.6, 30.5], [1...|          114.0|           40.9|
|03370f68-42e1-4b7...|[[100.7, 39.8], [...|           90.1|           42.2|
|047d84cd-d7a4-42c...|[[102.8, 45.7], [...|           95.8|           33.1|
|04a2e3f4-76d7-473...|[[114.2, 40.3], [...|          113.5|           42.1|
|0578985c-8d29-45c...|[[78.4, 20.8], [1...|          100.5|           48.1|
|0609f8a7-afb0-4ea...|[[113.2, 33.1], [...|          110.9|           46.0|
|067e382d-f2eb-4fe...|[[96.3, 42.1], [9...|           96.0|           43.9|
|071e138d-34b1-44b...|[[103.0, 16.4], [...|          114.1|           41.6|
|0751ce86-9d8f-48c...|[[110.3, 38.9], [...|          112.1|           48.0|
|07f9d02a-9add-427...|[[103.5, 62.0], [...|           96.0|           45.7|
|086a8d94-d8

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType  # Correct import for IntegerType

goal_X = 120
goal_Y1, goal_Y2 = 36, 44

def calculate_area(x1, y1, x2, y2, x3, y3):
    return abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)) / 2.0

def is_point_inside_triangle(player_x,player_y,shot_x,shot_y):
    # Total area of the triangle ABC
    area_abc = calculate_area(shot_x,shot_y,goal_X,goal_Y1,goal_X,goal_Y2)

    # Area of sub-triangles ABP, BCP, CAP
    area_abp = calculate_area(shot_x,shot_y,goal_X,goal_Y1,player_x,player_y)
    area_bcp = calculate_area(shot_x,shot_y,goal_X,goal_Y2,player_x,player_y)
    area_cap = calculate_area(goal_X,goal_Y2,goal_X,goal_Y1,player_x,player_y)

    return np.isclose(area_abc, (area_abp + area_bcp + area_cap))

# Define a PySpark UDF to check if a point is inside the triangle
def check_point_inside(coordinates, shot_x, shot_y):
    count = 0
    for coord in coordinates:
        player_x, player_y = coord
        if is_point_inside_triangle(player_x, player_y, shot_x, shot_y):
            count += 1
    return count

# Register the UDF
check_point_udf = F.udf(check_point_inside, returnType=IntegerType())  # Use IntegerType from pyspark.sql.types

# Add a column with the count of points inside the area for each row in the DataFrame
frames = frames.withColumn("points_inside_area", check_point_udf(F.col("coordinates"), F.col("shot_location_x"), F.col("shot_location_y")))

# Show the resulting DataFrame
frames.show()

24/12/23 22:47:10 WARN TaskSetManager: Stage 9 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+--------------------+--------------------+---------------+---------------+------------------+
|             Shot_id|         coordinates|shot_location_x|shot_location_y|points_inside_area|
+--------------------+--------------------+---------------+---------------+------------------+
|00d83f31-e271-4a1...|[[96.6, 30.5], [1...|          114.0|           40.9|                 1|
|03370f68-42e1-4b7...|[[100.7, 39.8], [...|           90.1|           42.2|                 4|
|047d84cd-d7a4-42c...|[[102.8, 45.7], [...|           95.8|           33.1|                 2|
|04a2e3f4-76d7-473...|[[114.2, 40.3], [...|          113.5|           42.1|                 1|
|0578985c-8d29-45c...|[[78.4, 20.8], [1...|          100.5|           48.1|                 2|
|0609f8a7-afb0-4ea...|[[113.2, 33.1], [...|          110.9|           46.0|                 2|
|067e382d-f2eb-4fe...|[[96.3, 42.1], [9...|           96.0|           43.9|                 1|
|071e138d-34b1-44b...|[[103.0, 16.4], [...|       

In [15]:
frames.orderBy('points_inside_area',ascending=False).select('Shot_id','shot_location_x','shot_location_y','points_inside_area').show(truncate=False)

24/12/23 22:48:03 WARN TaskSetManager: Stage 21 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------------+---------------+---------------+------------------+
|Shot_id                             |shot_location_x|shot_location_y|points_inside_area|
+------------------------------------+---------------+---------------+------------------+
|cbac1a62-8213-4f63-a123-f5ff268d4905|90.4           |56.1           |13                |
|aa569f84-10b2-4fa3-9e13-8efed921fe79|98.6           |42.3           |13                |
|02677ab3-4102-4223-9845-b0e9e1b29d5d|98.6           |41.0           |12                |
|df6b62f2-83a9-4891-9e60-a0640d0ec1e3|94.1           |31.9           |11                |
|2043dfb1-d2c9-4789-907f-028313c484b6|103.5          |39.0           |11                |
|c89fc6af-b522-483d-b046-3f89810c6be5|100.7          |40.4           |11                |
|034b3443-2699-427a-b4d7-df552855c1fb|110.4          |48.7           |11                |
|5ec1d57c-bf9f-4433-9666-b290929a88db|104.0          |36.0           |10                |
|6d3f823d-

In [16]:
events_shot = events_shot.join(frames.select('Shot_id','points_inside_area'),events_shot.id == frames.Shot_id).drop('Shot_id')

In [24]:
events_shot.orderBy('points_inside_area',ascending=False).show(truncate=False)

24/12/23 23:03:16 WARN TaskSetManager: Stage 61 contains a task of very large size (1681 KiB). The maximum recommended task size is 1000 KiB.


+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+------------------+
|id                                  |player_id|shot_location_x|shot_location_y|distance_to_goal|shot_angle|preferred_foot_shot|under_pressure|shot_aerial_won|shot_first_time|shot_one_on_one|shot_open_goal|shot_follows_dribble|shot_statsbomb_xg|shot_outcome|goal|points_inside_area|
+------------------------------------+---------+---------------+---------------+----------------+----------+-------------------+--------------+---------------+---------------+---------------+--------------+--------------------+-----------------+------------+----+------------------+
|cbac1a62-8213-4f63-a123-f5ff268d4905|13126.0  |90.4           |56.1           |33.6952         |11.944677 |1                  |0             |0       